# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [70]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [71]:
cities_file = "../WeatherPy/output_data/cities.csv"
cities_file_df = pd.read_csv(cities_file)
cities_file_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Port Macquarie,-31.43,152.92,19.44,66,0,0.45,AU,1596346756
1,Pitkyaranta,61.57,31.48,16.21,66,78,0.75,RU,1596346756
2,George Town,5.41,100.34,31.00,74,20,2.10,MY,1596346697
3,Guanaja,16.40,-85.90,28.44,78,8,2.35,HN,1596346756
4,Arlit,18.74,7.39,30.51,48,41,1.63,NE,1596346756


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [84]:
gmaps.configure(api_key = g_key)
locations = cities_file_df[['Lat', 'Lng']]
humidity =  cities_file_df['Humidity']

In [83]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = float(humidity.max()), point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.
* These are my conditions: Max temperature higher than 25 degrees C, wind speed less than 3 meter/sec and zero cloudiness.

In [85]:
vacation_cities_df = cities_file_df[(cities_file_df['Max Temp'] > 25) & (cities_file_df['Wind Speed'] < 2) & (cities_file_df['Cloudiness'] == 0)]
vacation_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
258,Cabo San Lucas,22.89,-109.91,30.56,60,0,0.45,MX,1596346639
281,Borogontsy,62.67,131.17,27.60,25,0,1.68,RU,1596346835
282,Ashquelon,31.67,34.57,26.11,58,0,1.34,IL,1596346836
353,Khāsh,28.22,61.22,34.51,23,0,1.41,IR,1596346855
368,Tabas,33.60,56.92,37.00,16,0,1.51,IR,1596346859
398,Kiryat Gat,31.61,34.76,26.11,58,0,1.34,IL,1596346868
541,São Filipe,14.90,-24.50,25.35,83,0,0.50,CV,1596346910


In [86]:
vacation_by_cities = vacation_cities_df.set_index('City')
vacation_by_cities

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City,,,,,,,,
Cabo San Lucas,22.89,-109.91,30.56,60,0,0.45,MX,1596346639
Borogontsy,62.67,131.17,27.60,25,0,1.68,RU,1596346835
Ashquelon,31.67,34.57,26.11,58,0,1.34,IL,1596346836
Khāsh,28.22,61.22,34.51,23,0,1.41,IR,1596346855
Tabas,33.60,56.92,37.00,16,0,1.51,IR,1596346859
Kiryat Gat,31.61,34.76,26.11,58,0,1.34,IL,1596346868
São Filipe,14.90,-24.50,25.35,83,0,0.50,CV,1596346910


In [87]:
#Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates
hotel_df = pd.DataFrame()
for city in vacation_by_cities.index:
    target_coordinates = str(vacation_by_cities.loc[city]['Lat']) + ', ' + str(vacation_by_cities.loc[city]['Lng'])
    target_radius = 5000
    target_type = 'hotel'

    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params)
    hotel_data = response.json()
    hotel_dict = {
        'Hotel Name': hotel_data["results"][1]["name"],
        'Address': hotel_data["results"][1]["vicinity"],
        'City': hotel_data["results"][0]["vicinity"],
        'Country': vacation_by_cities.loc[city]['Country'],
        'Lat': hotel_data["results"][1]["geometry"]["location"]["lat"],
        'Lng': hotel_data["results"][1]["geometry"]["location"]["lng"]
    }
    hotel_df = hotel_df.append(hotel_dict, ignore_index=True)
hotel_df

,Address,City,Country,Hotel Name,Lat,Lng
0,"Boulevard Paseo de la Marina LT 9 y 10, Marina...",Cabo San Lucas,MX,Hotel Tesoro Los Cabos,22.880938,-109.911797
1,"Героя Егорова 8, Borogontsy",Borogontsy,RU,"Mbu ""Ust'-Aldanskoye Upravleniye Kul'tury""",62.669807,131.165309
2,"Golani Street 11, Ashkelon",Ashkelon,IL,Leonardo Ashkelon,31.682938,34.561614
3,"Khash, خیابان رودکی",Khash,IR,Golestan Khash Internet Cafe,28.218361,61.218443
4,"Tabas, Imam Khomeini Boulevard",Tabas,IR,کافی نت طبس آنلاین,33.595972,56.928450
5,Kiryat Gat,Kiryat Gat,IL,Tel Aerni Tel Jet,31.611589,34.784876
6,"Praia Nossa Senhora da Encarnação, São Filipe ...",Sao Filipe,CV,Tortuga B&B,14.880736,-24.492962


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [88]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [90]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))